In [1]:
import time 
time.time()
len('1500000100042262992')

19

In [1]:
import sys
sys.path.append('..')

In [2]:
import CAT
import json
import torch
import logging
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorboardX import SummaryWriter

In [3]:
def setuplogger():
    root = logging.getLogger()
    root.setLevel(logging.INFO)
    handler = logging.StreamHandler(sys.stdout)
    handler.setLevel(logging.INFO)
    formatter = logging.Formatter("[%(levelname)s %(asctime)s] %(message)s")
    handler.setFormatter(formatter)
    root.addHandler(handler)
setuplogger()

In [4]:
seed = 0
np.random.seed(seed)
torch.manual_seed(seed)

In [5]:
# tensorboard
log_dir = f"../logs/{datetime.datetime.now().strftime('%Y-%m-%d-%H:%M')}/"
print(log_dir)
writer = SummaryWriter(log_dir)

../logs/2022-06-23-15:51/


In [6]:
# choose dataset here
dataset = 'junyi'
# modify config here
config = {
    'learning_rate': 0.0025,
    'batch_size': 2048,
    'num_epochs': 8,
    'num_dim': 1, # for IRT or MIRT
    'device': 'cpu',
    # for NeuralCD
    'prednet_len1': 128,
    'prednet_len2': 64,
}
# modify checkpoint path here
ckpt_path = '../ckpt/junyi/ncd.pt'

In [7]:
# read datasets
test_triplets = pd.read_csv(f'../data/{dataset}/test_triples.csv', encoding='utf-8').to_records(index=False)
concept_map = json.load(open(f'../data/{dataset}/item_topic.json', 'r'))
concept_map = {int(k):v for k,v in concept_map.items()}
metadata = json.load(open(f'../data/{dataset}/metadata.json', 'r'))

In [8]:
test_data = CAT.dataset.AdapTestDataset(test_triplets, concept_map,
                                        metadata['num_test_students'], 
                                        metadata['num_questions'], 
                                        metadata['num_concepts'])

In [9]:
import warnings
warnings.filterwarnings("ignore")

In [11]:
import torch.utils.data as data
# train_loader = data.DataLoader(test_data, batch_size=config['batch_size'], shuffle=True)

# fixed test length
test_length = 20
# choose strategies here
# strategies = [CAT.strategy.RandomStrategy(), CAT.strategy.MFIStrategy(), CAT.strategy.KLIStrategy()]
# , CAT.strategy.MAATStrategy()]
strategies = [CAT.strategy.RandomStrategy()]
df = pd.DataFrame() 
for strategy in strategies:
    model = CAT.model.NCDModel(**config)
    model.init_model(test_data)
    model.adaptest_load(ckpt_path)
    test_data.reset()
    
    logging.info('-----------')
    logging.info(f'start adaptive testing with {strategy.name} strategy')

    logging.info(f'Iteration 0')
    # evaluate models
    print(test_data.data)
    results = model.eval(test_data.data,config['device'])
    res_df=pd.DataFrame([[item for item in results.values()]],columns=['ACC','AUC','Coverage'])
    # for name, value in results.items():
    #     logging.info(f'{name}:{value}')
    
    # for it in range(1, test_length+ 1):
    #     logging.info(f'Iteration {it}')
    #     # select question
    #     selected_questions = strategy.adaptest_select(model, test_data)
    #     for student, question in selected_questions.items():
    #         test_data.apply_selection(student, question)
    #     # update models
    #     model.adaptest_update(test_data)
    #     # evaluate models
    #     results = model.eval(test_data,config['device'])
    #     # log results
    #     for name, value in results.items():
    #         logging.info(f'{name}:{value}')
    #         writer.add_scalars(name, {strategy.name: value}, it)
    #     tmp_df=pd.DataFrame([[item for item in results.values()]],columns=['ACC','AUC','Coverage'])
    #     res_df = res_df.append(tmp_df, ignore_index=True)
    # res_df = res_df.transpose()
    # df = df.append(res_df)
    # # df = df.append(pd.Series(),ignore_index=True)
    # df =df.append(pd.Series(name = ' '))
df.to_csv(f"../data/{dataset}/model/ncd/{datetime.datetime.now().strftime('%Y-%m-%d-%H:%M')}.csv")

[INFO 2022-06-23 15:52:22,034] -----------
[INFO 2022-06-23 15:52:22,035] start adaptive testing with Random Select Strategy strategy
[INFO 2022-06-23 15:52:22,036] Iteration 0
{0: {181: 1, 587: 1, 78: 0, 81: 0, 82: 0, 83: 1, 593: 1, 86: 1, 87: 1, 585: 0, 88: 1, 91: 1, 603: 0, 604: 1, 606: 0, 96: 0, 608: 0, 98: 1, 609: 0, 601: 0, 610: 0, 584: 0, 73: 1, 556: 1, 557: 1, 560: 0, 562: 0, 563: 0, 564: 1, 566: 1, 56: 0, 74: 1, 568: 1, 575: 0, 64: 0, 578: 0, 67: 1, 68: 1, 580: 1, 582: 0, 72: 0, 62: 0, 44: 0, 101: 0, 103: 1, 469: 0, 468: 0, 467: 0, 466: 1, 463: 0, 461: 1, 460: 0, 459: 1, 472: 0, 457: 0, 452: 1, 451: 1, 450: 0, 447: 0, 446: 0, 442: 0, 441: 0, 435: 1, 455: 1, 612: 1, 474: 0, 482: 0, 613: 0, 105: 1, 615: 1, 511: 1, 510: 0, 505: 0, 503: 0, 502: 0, 480: 0, 498: 0, 494: 1, 493: 0, 492: 0, 491: 0, 488: 1, 487: 0, 486: 0, 485: 0, 495: 1, 430: 0, 555: 0, 41: 0, 479: 1, 478: 1, 484: 1, 448: 1, 437: 1, 438: 1, 439: 0, 440: 1, 518: 1, 443: 1, 445: 1, 477: 0, 449: 1, 453: 1, 436: 1, 454: 1

evaluating:   0%|          | 0/23 [00:00<?, ?it/s]


TypeError: 'numpy.int64' object is not iterable